In [ ]:
##The following scripts run in google earth engine platform

In [ ]:
Map.addLayer(roi, {}, 'ROI')
Map.centerObject(roi, 8)
//Load Sentinel-1 SAR collection and filter according to data collection type
var S1 = ee.ImageCollection('COPERNICUS/S1_GRD')
  .filterBounds(roi)
  .filterDate('2022-01-01','2022-12-31')
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))

//Add first image to map to get an idea of what a SAR image looks like
Map.addLayer(S1.first(),{bands: 'VV',min: -18, max: 0}, 'SAR image')

// Filter speckle noise
var filterSpeckles = function(img) {
  var vv = img.select('VV') //select the VV polarization band
  var vv_smoothed = vv.focal_median(100,'circle','meters').rename('VV_Filtered') //Apply a focal median filter
  return img.addBands(vv_smoothed) // Add filtered VV band to original image
}

// Map speckle noise filter across collection. Result is same collection, with smoothed VV band added to each image
S1 = S1.map(filterSpeckles)

//Add speckle filtered image to map to sompare with raw SAR image
Map.addLayer(S1.first(),{bands: 'VV_Filtered',min: -18, max: 0}, 'Filtered SAR image')
//Here we are using -16. This is only an approximation and will result in some errors. Try adjusting the
var classifyWater = function(img) {
  var vv = img.select('VV_Filtered')
  var water = vv.lt(-16).rename('Water')  //Identify all pixels below threshold and set them equal to 1. All other pixels set to 0
  water = water.updateMask(water) //Remove all pixels equal to 0
  return img.addBands(water)  //Return image with added classified water band
}

//Map classification across sentinel-1 collection and print to console to inspect
S1 = S1.map(classifyWater)
print(S1)
//Make time series of water pixels within region
var ClassChart = ui.Chart.image.series({
  imageCollection: S1.select('Water'),
  region: roi,
  reducer: ee.Reducer.sum(),
  scale: 100,
})
  .setOptions({
      title: 'Inundated Pixels',
      hAxis: {'title': 'Date'},
      vAxis: {'title': 'Number of Inundated Pixels'},
      lineWidth: 2
    })

//Set the postion of the chart and add it to the map
ClassChart.style().set({
    position: 'bottom-right',
    width: '500px',
    height: '300px'
  });

Map.add(ClassChart)
// Create a label on the map.
var label = ui.Label('Click a point on the chart to show the image for that date.');
Map.add(label);
//Create callbakc function that adds image to the map coresponding with clicked data point on chart
ClassChart.onClick(function(xValue, yValue, seriesName) {
    if (!xValue) return;  // Selection was cleared.

    // Show the image for the clicked date.
    var equalDate = ee.Filter.equals('system:time_start', xValue);
    //Find image coresponding with clicked data and clip water classification to roi
    var classification = ee.Image(S1.filter(equalDate).first()).clip(roi).select('Water');
    var SARimage = ee.Image(S1.filter(equalDate).first());
    //Make map layer based on SAR image, reset the map layers, and add this new layer
    var S1Layer = ui.Map.Layer(SARimage, {
      bands: ['VV'],
      max: 0,
      min: -20
    });
    Map.layers().reset([S1Layer]);
    var visParams = {
      min: 0,
      max: 1,
      palette: ['#FFFFFF','#0000FF']
    }
    //Add water classification on top of SAR image
    Map.addLayer(classification,visParams,'Water')

    // Show a label with the date on the map.
    label.setValue((new Date(xValue)).toUTCString());
  });
  // Load Sentinel-1 SAR collection and filter according to data collection type
Map.addLayer(roi, {}, 'ROI');
Map.centerObject(roi, 8);

var S1 = ee.ImageCollection('COPERNICUS/S1_GRD')
  .filterBounds(roi)
  .filterDate('2022-01-01', '2022-12-31')
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'));

// Add first image to map to get an idea of what a SAR image looks like
Map.addLayer(S1.first(), {bands: 'VV', min: -18, max: 0}, 'SAR image');

// Speckle noise filtering function
var filterSpeckles = function(img) {
  var vv = img.select('VV');
  var vv_smoothed = vv.focal_median(100, 'circle', 'meters').rename('VV_Filtered');
  return img.addBands(vv_smoothed);
};
S1 = S1.map(filterSpeckles);

// Add filtered image to compare with raw SAR image
Map.addLayer(S1.first(), {bands: 'VV_Filtered', min: -18, max: 0}, 'Filtered SAR image');

// Water classification function
var classifyWater = function(img) {
  var vv = img.select('VV_Filtered');
  var water = vv.lt(-16).rename('Water');
  water = water.updateMask(water);
  return img.addBands(water);
};
S1 = S1.map(classifyWater);
print(S1, 'Classified SAR Collection');

// Time series chart of water pixels
var ClassChart = ui.Chart.image.series({
  imageCollection: S1.select('Water'),
  region: roi,
  reducer: ee.Reducer.sum(),
  scale: 100,
}).setOptions({
  title: 'Inundated Pixels',
  hAxis: {title: 'Date'},
  vAxis: {title: 'Number of Inundated Pixels'},
  lineWidth: 2
});
ClassChart.style().set({
  position: 'bottom-right',
  width: '500px',
  height: '300px'
});
Map.add(ClassChart);

var label = ui.Label('Click a point on the chart to show the image for that date.');
Map.add(label);

// OnClick function to show selected date SAR image
ClassChart.onClick(function(xValue, yValue, seriesName) {
  if (!xValue) return;
  var equalDate = ee.Filter.equals('system:time_start', xValue);
  var classification = ee.Image(S1.filter(equalDate).first()).clip(roi).select('Water');
  var SARimage = ee.Image(S1.filter(equalDate).first());

  var S1Layer = ui.Map.Layer(SARimage, {
    bands: ['VV'],
    max: 0,
    min: -20
  });
  Map.layers().reset([S1Layer]);

  var visParams = {
    min: 0,
    max: 1,
    palette: ['#FFFFFF', '#0000FF']
  };
  Map.addLayer(classification, visParams, 'Water');
  label.setValue((new Date(xValue)).toUTCString());
});

// -------- ADDITION: Change Detection Mapping -------- //

// Step 1: Select two time periods (pre and post event)
var preEvent = S1.filterDate('2022-03-01', '2022-06-01').median();
var postEvent = S1.filterDate('2022-09-01', '2022-12-31').median();

// Step 2: Extract water classification for both periods
var preWater = preEvent.select('Water').rename('Water_Pre');
var postWater = postEvent.select('Water').rename('Water_Post');

// Step 3: Calculate change detection by subtracting pre from post
var waterChange = postWater.subtract(preWater).rename('Water_Change');

// Step 4: Visualization parameters
var changeVis = {
  min: -1,
  max: 1,
  palette: ['#FF0000', '#FFFFFF', '#0000FF']  // Red = loss, Blue = gain, White = no change
};

// Step 5: Display results
Map.addLayer(preWater, {palette: 'blue'}, 'Pre-Event Water');
Map.addLayer(postWater, {palette: 'blue'}, 'Post-Event Water');
Map.addLayer(waterChange, changeVis, 'Water Change Detection');

// Step 6: Export change map to Google Drive
Export.image.toDrive({
  image: waterChange,
  description: 'Water_Change_Detection',
  scale: 10,
  region: roi,
  fileFormat: 'GeoTIFF'
});
// ------------------------- Step 1: Use Existing ROI -------------------------
// Use the already defined ROI from the previous script
Map.addLayer(roi, {}, 'ROI');
Map.centerObject(roi, 8);

// Load Sentinel-1 SAR Collection
var S1 = ee.ImageCollection('COPERNICUS/S1_GRD')
  .filterBounds(roi)
  .filterDate('2022-01-01', '2022-12-31')
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
  .select('VV');

// ------------------------- Step 2: Apply Speckle Filtering -------------------------
var filterSpeckles = function(img) {
  var vv = img.select('VV');
  var vv_smoothed = vv.focal_median(100, 'circle', 'meters').rename('VV_Filtered');
  return img.addBands(vv_smoothed);
};
S1 = S1.map(filterSpeckles);

// ------------------------- Step 3: Create Training Data -------------------------
// Define Water and Non-Water Points (Manually or using existing data)
var water = ee.FeatureCollection([
  ee.Feature(roi, {class: 1})  // Water points
]);
var nonWater = ee.FeatureCollection([
  ee.Feature(roi, {class: 0})  // Non-water points
]);

// Merge training points
var trainingPoints = water.merge(nonWater);

// Sample Sentinel-1 for training
var training = S1.first().sampleRegions({
  collection: trainingPoints,
  properties: ['class'],
  scale: 30
});

// ------------------------- Step 4: Train Random Forest and GBT Models -------------------------
// Train Random Forest Classifier
var rfClassifier = ee.Classifier.smileRandomForest(100).train({
  features: training,
  classProperty: 'class'
});

// Train Gradient Boosted Trees Classifier
var gbtClassifier = ee.Classifier.smileGradientTreeBoost(50).train({
  features: training,
  classProperty: 'class'
});

// ------------------------- Step 5: Apply Classifiers to Entire Image Collection -------------------------
var classifyImages = function(image) {
  var rfClassified = image.classify(rfClassifier).rename('RF_Classification');
  var gbtClassified = image.classify(gbtClassifier).rename('GBT_Classification');
  return image.addBands([rfClassified, gbtClassified]);
};

var classified = S1.map(classifyImages);
print(classified, 'Classified SAR Collection');

// ------------------------- Step 6: Perform Change Detection -------------------------
// Select Pre-Event and Post-Event Time Periods
var preEvent = classified.filterDate('2022-03-01', '2022-06-01').median();
var postEvent = classified.filterDate('2022-09-01', '2022-12-31').median();

// Extract Random Forest Classifications
var preRF = preEvent.select('RF_Classification').rename('Pre_RF');
var postRF = postEvent.select('RF_Classification').rename('Post_RF');
var rfChange = postRF.subtract(preRF).rename('RF_Change');

// Extract GBT Classifications
var preGBT = preEvent.select('GBT_Classification').rename('Pre_GBT');
var postGBT = postEvent.select('GBT_Classification').rename('Post_GBT');
var gbtChange = postGBT.subtract(preGBT).rename('GBT_Change');

// ------------------------- Step 7: Visualize Change Detection -------------------------
var visParams = {
  min: -1,
  max: 1,
  palette: ['red', 'white', 'blue']  // Red = Loss, Blue = Gain, White = No Change
};

// Add Layers to Map
Map.addLayer(preRF, {min: 0, max: 1, palette: ['white', 'blue']}, 'Pre-Event (RF)');
Map.addLayer(postRF, {min: 0, max: 1, palette: ['white', 'blue']}, 'Post-Event (RF)');
Map.addLayer(rfChange, visParams, 'RF Change Detection');

Map.addLayer(preGBT, {min: 0, max: 1, palette: ['white', 'blue']}, 'Pre-Event (GBT)');
Map.addLayer(postGBT, {min: 0, max: 1, palette: ['white', 'blue']}, 'Post-Event (GBT)');
Map.addLayer(gbtChange, visParams, 'GBT Change Detection');

// ------------------------- Step 8: Export Results -------------------------
Export.image.toDrive({
  image: rfChange,
  description: 'RF_Change_Detection',
  scale: 10,
  region: roi,
  fileFormat: 'GeoTIFF'
});

Export.image.toDrive({
  image: gbtChange,
  description: 'GBT_Change_Detection',
  scale: 10,
  region: roi,
  fileFormat: 'GeoTIFF'
});
// ------------------------- Step 1: Split Training and Testing Data -------------------------

// Merge water and non-water points for labeled dataset
var labeledData = water.merge(nonWater);

// Randomly split 70% for training, 30% for testing
var split = 0.7;
var trainingData = labeledData.randomColumn('random').filter(ee.Filter.lt('random', split));
var testingData = labeledData.randomColumn('random').filter(ee.Filter.gte('random', split));

// Visualize the split data on the map
Map.addLayer(trainingData, {color: 'blue'}, 'Training Data');
Map.addLayer(testingData, {color: 'red'}, 'Testing Data');

// ------------------------- Step 2: Train Random Forest and GBT Models -------------------------

// Sample from Sentinel-1 SAR for training
var trainingSamples = S1.first().sampleRegions({
  collection: trainingData,
  properties: ['class'],
  scale: 30
});

// Train Random Forest Classifier
var rfClassifier = ee.Classifier.smileRandomForest(100).train({
  features: trainingSamples,
  classProperty: 'class'
});

// Train GBT Classifier
var gbtClassifier = ee.Classifier.smileGradientTreeBoost(50).train({
  features: trainingSamples,
  classProperty: 'class'
});

// ------------------------- Step 3: Apply Classifiers to Testing Data -------------------------

// Sample the testing data from SAR for validation
var testingSamples = S1.first().sampleRegions({
  collection: testingData,
  properties: ['class'],
  scale: 30
});

// Classify the testing samples
var rfValidation = testingSamples.classify(rfClassifier);
var gbtValidation = testingSamples.classify(gbtClassifier);

// ------------------------- Step 4: Calculate Confusion Matrices -------------------------

// Generate Confusion Matrices
var rfConfusionMatrix = rfValidation.errorMatrix('class', 'classification');
var gbtConfusionMatrix = gbtValidation.errorMatrix('class', 'classification');

// Print the Confusion Matrices to Console
print('Random Forest Confusion Matrix:', rfConfusionMatrix);
print('GBT Confusion Matrix:', gbtConfusionMatrix);

// ------------------------- Step 5: Compute Overall Accuracy and Kappa -------------------------

// RF Accuracy Metrics
print('Random Forest Overall Accuracy:', rfConfusionMatrix.accuracy());
print('Random Forest Kappa Coefficient:', rfConfusionMatrix.kappa());

// GBT Accuracy Metrics
print('GBT Overall Accuracy:', gbtConfusionMatrix.accuracy());
print('GBT Kappa Coefficient:', gbtConfusionMatrix.kappa());
------------------------- Step 1: Define ROI and Initialize -------------------------
------------------------- Use Existing ROI -------------------------
Load and visualize the ROI
Map.addLayer(roi, {}, 'ROI');
Map.centerObject(roi, 8);

// Load Sentinel-1 SAR Collection
var S1 = ee.ImageCollection('COPERNICUS/S1_GRD')
  .filterBounds(roi)
  .filterDate('2022-01-01', '2022-12-31')
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
  .select('VV');

// ------------------------- Speckle Filtering -------------------------
var filterSpeckles = function(img) {
  var vv = img.select('VV');
  var vv_smoothed = vv.focal_median(50, 'circle', 'meters').rename('VV_Filtered'); // Adjusted for memory
  return img.addBands(vv_smoothed);
};
S1 = S1.map(filterSpeckles);

// ------------------------- Automatic Sampling -------------------------
// Generate water and non-water masks using a threshold
var threshold = -16; // Approximation threshold for water
var waterMask = S1.median().select('VV_Filtered').lt(threshold).selfMask();
var nonWaterMask = S1.median().select('VV_Filtered').gte(threshold).selfMask();

// Sample points from the masks within the ROI
var waterSamples = waterMask.sample({
  region: roi,
  scale: 30,
  numPixels: 500, // Adjust as needed for balance and memory
  geometries: true
}).map(function(feature) {
  return feature.set('class', 1); // Label as water
});

var nonWaterSamples = nonWaterMask.sample({
  region: roi,
  scale: 30,
  numPixels: 500,
  geometries: true
}).map(function(feature) {
  return feature.set('class', 0); // Label as non-water
});

// Merge the water and non-water samples
var trainingSamples = waterSamples.merge(nonWaterSamples);

// Visualize sampled points
Map.addLayer(trainingSamples, {color: 'yellow'}, 'Training Samples');

// ------------------------- Train Classifier -------------------------
var training = S1.first().sampleRegions({
  collection: trainingSamples,
  properties: ['class'],
  scale: 30
});

var rfClassifier = ee.Classifier.smileRandomForest(50).train({
  features: training,
  classProperty: 'class'
});

// ------------------------- Apply Classifier -------------------------
var classifyImages = function(image) {
  var rfClassified = image.classify(rfClassifier).rename('RF_Classification');
  return image.addBands(rfClassified);
};

S1 = S1.map(classifyImages);

// ------------------------- Time Series Chart -------------------------
var chart = ui.Chart.image.series({
  imageCollection: S1.select('RF_Classification'),
  region: roi,
  reducer: ee.Reducer.mean(),
  scale: 100
}).setOptions({
  title: 'Water Classification Over Time',
  hAxis: {title: 'Date'},
  vAxis: {title: 'Mean Classification Value'},
});

Map.add(chart);

// ------------------------- Change Detection -------------------------
var preEvent = S1.filterDate('2022-03-01', '2022-06-01').median();
var postEvent = S1.filterDate('2022-09-01', '2022-12-31').median();

var preWater = preEvent.select('RF_Classification').rename('Water_Pre');
var postWater = postEvent.select('RF_Classification').rename('Water_Post');

var waterChange = postWater.subtract(preWater).rename('Water_Change');
Map.addLayer(waterChange, {
  min: -1,
  max: 1,
  palette: ['red', 'white', 'blue']
}, 'Water Change Detection');

// ------------------------- Export Results -------------------------
Export.image.toDrive({
  image: waterChange,
  description: 'Water_Change_Detection',
  scale: 10,
  region: roi,
  fileFormat: 'GeoTIFF'
});
// ------------------------- Train and Apply Random Forest -------------------------
var training = S1.first().sampleRegions({
  collection: trainingSamples,
  properties: ['class'],
  scale: 30
});

var rfClassifier = ee.Classifier.smileRandomForest(50).train({
  features: training,
  classProperty: 'class'
});

// Classify images using Random Forest
var classifyImagesRF = function(image) {
  var rfClassified = image.classify(rfClassifier).rename('RF_Classification');
  return image.addBands(rfClassified);
};
S1 = S1.map(classifyImagesRF);

// ------------------------- Accuracy Assessment for RF -------------------------
var validation = training.randomColumn('random', 0).filter(ee.Filter.lt('random', 0.3)); // 30% validation
var rfValidation = validation.classify(rfClassifier);

var rfAccuracy = rfValidation.errorMatrix('class', 'classification');
print('Random Forest Error Matrix:', rfAccuracy);
print('Random Forest Accuracy:', rfAccuracy.accuracy());

// Visualize RF Results
Map.addLayer(S1.median().select('RF_Classification'), {
  min: 0, max: 1, palette: ['yellow', 'blue']
}, 'RF Classification');

// ------------------------- Prepare for Gradient Boosting -------------------------
// Export training data for Gradient Boosting
Export.table.toDrive({
  collection: training,
  description: 'Training_Data_Export',
  fileFormat: 'CSV'
});

// Placeholder for Gradient Boosting Results
// - Train Gradient Boosting in Python or R (e.g., XGBoost, LightGBM, Scikit-learn).
// - Use exported CSV for input and calculate accuracy metrics externally.
// - Export classified results back to Earth Engine as GeoTIFF or use GEE for visualization.

// ------------------------- Visualization & Comparison -------------------------
// Add placeholders for imported GB results
var gbClassified = ee.Image('users/username/gb_classified_image'); // Import your GB classification results

Map.addLayer(gbClassified, {
  min: 0, max: 1, palette: ['yellow', 'green']
}, 'GB Classification');


